In [84]:
import pandas as pd
from bs4 import BeautifulSoup
import untangle
import xmltodict
import numpy as np

In [35]:
with open('/Users/meshchd/Downloads/TDM-08-05-19-response.xml') as f:
    soup = BeautifulSoup(f.read(), 'lxml')

In [353]:
print(soup.prettify)

<bound method Tag.prettify of <?xml version="1.0" encoding="UTF-8"?><html><body><taxonomydefinitionmessage xmlns="http://schemas.umusic.com/umg.eai.taxonomy-messages.v1" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
<messageheader xmlns="">
<messageid>8b681d64-f49e-4f80-ac7b-b8b66b47aeeb</messageid>
<messagesource>Amplify</messagesource>
<messageutcdatetimestamp>2019-05-08T23:24:48.8035961Z</messageutcdatetimestamp>
<resourcetype>TaxonomyDefinitionMessage</resourcetype>
<resourceid>9c0c39a8-c449-4f92-a61d-f30c3ca57ac5</resourceid>
<resourceutcdatetimestamp>2019-05-08T23:24:40.8295665Z</resourceutcdatetimestamp>
<schemaversion>1.0</schemaversion>
<defaultlanguagescriptcode>en</defaultlanguagescriptcode>
</messageheader>
<taxonomies xmlns="">
<taxonomy type="Recording">
<taxonomynodes>
<taxonomynode description="Sonic-cultural groupings of music." name="Genre" taggable="false" taxonomynodeid="78dfdd00-1fc1-47c6-888e-d8a376a0e44e">
<ta

In [46]:
taxonomynode = soup.find_all('taxonomynode')

In [349]:
item_parents = []
dict_list = []

for item in taxonomynode:
    item_attrs = dict(item.attrs)
    dict_list.append(item_attrs)
    
    test_parent = []
    for parent in item.parents:
        if 'name' in dict(parent.attrs).keys():
            test_parent.insert(0,dict(parent.attrs)['name'])
    item_parents.append(test_parent)

In [352]:
dict_list[:2]

[{'description': 'Sonic-cultural groupings of music.',
  'name': 'Genre',
  'taggable': 'false',
  'taxonomynodeid': '78dfdd00-1fc1-47c6-888e-d8a376a0e44e'},
 {'description': 'Characterized by a loose narrative lyrical style, use of call-and-response, the blues scale and blue notes, a small set of common chord progressions, and trance-like walking basslines. Originated in African-American communities in the Deep South of the United States in the late 19th century.',
  'name': 'Blues',
  'taggable': 'true',
  'taxonomynodeid': 'e6576040-ba5a-4f51-b547-3d1ec7005afd'}]

In [348]:
item_parents[:10]

[[],
 ['Genre'],
 ['Genre', 'Blues'],
 ['Genre', 'Blues', 'Modern Blues'],
 ['Genre', 'Blues', 'Modern Blues'],
 ['Genre', 'Blues', 'Modern Blues'],
 ['Genre', 'Blues', 'Modern Blues'],
 ['Genre', 'Blues'],
 ['Genre', 'Blues', 'Traditional Electric'],
 ['Genre', 'Blues', 'Traditional Electric']]

In [327]:
tags = pd.DataFrame(dict_list)
parents = pd.DataFrame(item_parents)
parents.columns = ['lvl_'+str(i) for i in range(parents.shape[1])]

In [343]:
print(parents.shape)
print(tags.shape)

(4341, 5)
(4341, 4)


In [344]:
tags.head()

,description,name,taggable,taxonomynodeid
0,Sonic-cultural groupings of music.,Genre,false,78dfdd00-1fc1-47c6-888e-d8a376a0e44e
1,Characterized by a loose narrative lyrical sty...,Blues,true,e6576040-ba5a-4f51-b547-3d1ec7005afd
2,NaN,Modern Blues,true,17be6648-0253-4759-b13c-63e0c7ea1c7e
3,British Blues refers to English bands that pla...,British Blues,true,4e35b1e9-2788-4088-9645-4adddd5eb0c7
4,Texas blues played with electric instruments.,Electric Texas Blues,true,cfb91212-419d-4e60-a833-a88ab30030da


In [354]:
parents.head()

,lvl_0,lvl_1,lvl_2,lvl_3,lvl_4
0,None,None,None,None,None
1,Genre,None,None,None,None
2,Genre,Blues,None,None,None
3,Genre,Blues,Modern Blues,None,None
4,Genre,Blues,Modern Blues,None,None


In [347]:
all_data = pd.merge(parents, tags, left_index=True, right_index=True)
all_data.head()

,lvl_0,lvl_1,lvl_2,lvl_3,lvl_4,description,name,taggable,taxonomynodeid
0,None,None,None,None,None,Sonic-cultural groupings of music.,Genre,false,78dfdd00-1fc1-47c6-888e-d8a376a0e44e
1,Genre,None,None,None,None,Characterized by a loose narrative lyrical sty...,Blues,true,e6576040-ba5a-4f51-b547-3d1ec7005afd
2,Genre,Blues,None,None,None,NaN,Modern Blues,true,17be6648-0253-4759-b13c-63e0c7ea1c7e
3,Genre,Blues,Modern Blues,None,None,British Blues refers to English bands that pla...,British Blues,true,4e35b1e9-2788-4088-9645-4adddd5eb0c7
4,Genre,Blues,Modern Blues,None,None,Texas blues played with electric instruments.,Electric Texas Blues,true,cfb91212-419d-4e60-a833-a88ab30030da


In [355]:
all_data.lvl_0.unique()

array([None, 'Genre', 'Energy', 'Anatomy', 'Application', 'Performance',
       'Lyrics', 'Origin', 'Emotion', 'Intensity', 'Tempo', 'Playlist',
       'Artist/Ensemble', 'Contents', 'Popularity', 'Event'], dtype=object)

In [357]:
all_data[all_data.lvl_0=='Playlist'].lvl_1.unique()

array([None, 'Activity', 'Location', 'Mood', 'Theme', 'Time'],
      dtype=object)

In [359]:
all_data[(all_data.lvl_0=='Playlist')&(all_data.lvl_1.isnull()==True)]

,lvl_0,lvl_1,lvl_2,lvl_3,lvl_4,description,name,taggable,taxonomynodeid
4160,Playlist,None,None,None,None,NaN,Activity,true,42f9d7e9-9133-4ddc-8a04-3c3b05a20ea8
4180,Playlist,None,None,None,None,NaN,Armageddon,true,de3b2728-d2be-41a3-90c3-302d4210aa7c
4181,Playlist,None,None,None,None,NaN,Circus,true,f39873e3-3f68-43eb-8c72-958c049f87eb
4182,Playlist,None,None,None,None,NaN,Graduation,true,75fc973a-1187-4408-a50f-cd21f9c711f6
4183,Playlist,None,None,None,None,NaN,Location,true,3ebac12f-bdd2-40a4-8f17-fe205dd52998
4205,Playlist,None,None,None,None,NaN,Mood,true,5c6464f0-48be-4367-a23e-7f37d669346f
4233,Playlist,None,None,None,None,NaN,Theme,true,e02cdb4a-8414-4dac-b47c-25069ead6661
4248,Playlist,None,None,None,None,NaN,Time,true,9b02de9f-3b17-4368-978d-c38a3cdbdd77
